# Indexing -> 
 1. Load: This involves loading our data and is done with the document loaders by langchain
 2. Split: Text Splitters break large (documents) into smaller chunks. This is practicle because large chunks are harder to search over and won't fit in the models
 3. Store: We need to store and index our splits so that they can be search over later, this is often done using a Vector Store and a embedding model.

In [ ]:
%pip install --quiet --upgrade pip
%pip install --quiet sentence-transformers==2.2.2
%pip install --quiet transformers torch
%pip install --quiet langchain-text-splitters langchain-community langgraph
%pip install --quiet -U langgraph "langchain[openai]"
%pip install --quiet openpyxl pandas unstructured msoffcrypto-tool
%pip install --quiet "unstructured[all-docs]" python-magic-bin
%pip install --quiet streamlit streamlit-chat
%pip install --quiet chromadb

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Verify all required packages are installed and working
import sys
import pkg_resources

required_packages = [
    'sentence_transformers',
    'transformers',
    'torch',
    'langchain',
    'langchain_community',
    'streamlit',
    'chromadb'
]

def check_package(package_name):
    try:
        pkg_resources.get_distribution(package_name)
        return True
    except pkg_resources.DistributionNotFound:
        return False

all_installed = True
for package in required_packages:
    is_installed = check_package(package)
    print(f"{package}: {'✅ Installed' if is_installed else '❌ Not installed'}")
    if not is_installed:
        all_installed = False

if not all_installed:
    print("\n❌ Some packages are missing. Please run the installation cell above again.")

In [3]:
# Langsmith to track and verify changes

import getpass
import os
from dotenv import load_dotenv

load_dotenv()

LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = LANGSMITH_API_KEY


In [4]:
from openai import OpenAI

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,
    default_headers={
        "HTTP-Referer": "https://github.com/Wayn-Git/INGRES-AI",  # Optional, for tracking only
        "X-Title": "INGRES-AI RAG"  # Optional, for tracking only
    }
)

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# Initialize the chat model with DeepSeek through OpenRouter
llm = ChatOpenAI(
    model="deepseek/deepseek-chat-v3.1:free",  # DeepSeek chat model
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=OPENROUTER_API_KEY,
    temperature=0.7,
    max_tokens=2000
)

# Test the model
response = llm.invoke([HumanMessage(content="Answer in one word, can you speak ?")])
print(response.content)

No.


In [6]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders.excel import UnstructuredExcelLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import glob
import os

# Find all Excel files in the data directory
excel_files = glob.glob(os.path.join('../data', '**/*.xlsx'), recursive=True)

# Initialize empty list for documents
documents = []

# Load each Excel file
for excel_file in excel_files:
    try:
        loader = UnstructuredExcelLoader(excel_file)
        documents.extend(loader.load())
        print(f"Successfully loaded: {excel_file}")
    except Exception as e:
        print(f"Error loading {excel_file}: {str(e)}")

# Initialize the text splitter with settings optimized for tabular data
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,  # Slightly larger chunks for table content
    chunk_overlap=250,  # More overlap to maintain context
    length_function=len,
    is_separator_regex=False,
    separators=["\n\n", "\n", " ", ""]  # Custom separators for Excel content
)

# Split the documents
splits = text_splitter.split_documents(documents)
print(f"\nSuccessfully loaded {len(documents)} Excel files and split them into {len(splits)} chunks")

Successfully loaded: ../data\CentralReport1758469494933.xlsx
Successfully loaded: ../data\CentralReport1758469537172.xlsx

Successfully loaded 2 Excel files and split them into 27 chunks


In [12]:
%pip install chromadb


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/19.8 MB ? eta -:--:--
    --------------------------------------- 0.3/19.8 MB ? eta -:--:--
   - -------------------------------------- 0.8/19.8 MB 2.1 MB/s eta 0:00:10
   -- ------------------------------------- 1.3/19.8 MB 2.1 MB/s eta 0:00:09
   ---- ----------------------------------- 2.1/19.8 MB 2.6 MB/s eta 0:00:07
   ----- ---------------------------------- 2.9/19.8 MB 2.9 MB/s eta 0:00:06
   ------- -------------------------------- 3.9/19.8 MB 3.3 MB/s eta 0:00:05
   ----------- ---------------------------- 5.5/19.8 MB 3.9 MB/s eta 0:00:04
   -------------- ------------------------- 7.1/19.8 MB 4.4 MB/s et

In [13]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma  # <- different vector store
import os

try:
    # Initialize embeddings using a smaller, more stable model
    embeddings = HuggingFaceEmbeddings(
        model_name="all-MiniLM-L6-v2",
        model_kwargs={'device': 'cpu'}
    )
    
    print("Successfully initialized embeddings model")
    
    # Create vector store using Chroma
    vectorstore = Chroma.from_documents(
        documents=splits,
        embedding=embeddings,
        persist_directory="chroma_index"  # folder to save the DB
    )
    
    print("Successfully created Chroma vector store")
    
    # Persist the vector store to disk
    vectorstore.persist()
    print("Successfully saved Chroma vector store")
    
    # Test a simple similarity search
    query = "What is this document about?"
    docs = vectorstore.similarity_search(query)
    print(f"\nFound {len(docs)} relevant documents")
    for i, doc in enumerate(docs, 1):
        print(f"\nDocument {i}:")
        print(doc.page_content[:200] + "...")
        
except Exception as e:
    print(f"An error occurred: {str(e)}")
    print("Please try restarting the kernel and running all cells in order")


Successfully initialized embeddings model
Successfully created Chroma vector store
Successfully saved Chroma vector store

Found 4 relevant documents

Document 1:
"Report" for : ANDAMAN AND NICOBAR ISLANDS for year 2024-2025

Assessment Year 2024-2025 Top View STATE View admin Computation Type normal...

Document 2:
"Report" for : ANDAMAN AND NICOBAR ISLANDS for year 2012-2013

Assessment Year 2012-2013 Top View STATE View admin Computation Type normal...

Document 3:
Recharges Lateral Flows Vertical Flows Evaporation Transpiration Evapotranspiration C NC PQ Total Domestic Industrial Irrigation C NC PQ Total Major Parameter Present Other Parameters Present C NC PQ ...

Document 4:
Recharges Lateral Flows Vertical Flows Evaporation Transpiration Evapotranspiration C NC PQ Total Domestic Industrial Irrigation C NC PQ Total Major Parameter Present Other Parameters Present C NC PQ ...
Successfully created Chroma vector store
Successfully saved Chroma vector store

Found 4 relevant document

C:\Users\bilal\AppData\Local\Temp\ipykernel_1696\4024723648.py:24: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()
